# Initial Model

## Variables
variable| definition
-|-
$l_n$| tide level at hour $n$
$h$| height of the fish trap wall
$r$| radius of the semicircular fish trap
$N$| number of fish in the area
$F_n$| number of fish not in trap at hour $n$
$C_n$| number of fish in the trap at hour $n$
$T_n$| total number of fish harvest at hour $n$
$\alpha$| constant of fish movement, the likelyhood a fish in the area would swim over the largest allowed semi-circle
$p_n$| ratio of perimiter of semi-circle covered by water at hour $n$ to the largest allowed circle


## Diagram
![title](resources/diagram.png)

## Assumptions

Assume that the ammount of amount of fish in the area at any given time is some constant $N$ regardless of the number of fish recently harvested. This seems reasonable for short periods of time and in the absense of industrial harvesting methods.

Fish are harvested when the water level is below the lowest point of the trap.

Fish movement patterns in the area is random.

The highest point of the trap is above or just below high tide level.

## Plan
To save either doing a crude approximation of a tide such as a simple cosine graph or a complex, but accurate tidal equation that is hidden to the end user (elementary aged children), use obtainable tide data from  comox harbour, a site of fish traps. Additionally, since the code an inner workings of the model are hidden from the end user we can use the hourly tidal information to build a discrete model that would have all appearances of being continuous and allow for the effect of user controlled variables to be apparent. 

![title](resources/tide.png)

Using a discrete method, at each hour step can multiply the number of fish in each catagory by the proportion of the semi circle that is covered by water and by some constant $\alpha \leq 1$. $\alpha$ can be adjusted until it 'feels' right so the children can be challenged in making their traps.

## Equations

The top ridge of the trap expressed as intersection of a half-cylinder and a plane: $x = r\cdot\cos(\theta)$, $y = r\cdot\sin(\theta)$, $z = 6 + h + (0.17 * y)$, for $h,r$: user input, $\theta\in(\pi,2\pi)$.

The $z$ values are from the slope of the shore: $z = 6 + (0.17 * y)$ with the h variable added by the user.

$p_n$:

To find the length of the perimeter of the semi-circle covered by water we find, moving in the x direction, the first point of the perimeter covered by water then working in second quadrant of the circle of radius $r$ solve for the angle to that point. We take the ratio of that angle with $pi/2$ and multiply it by the length of the total semi-cicle, $\pi r$, finally divide this by the largest allowed perimeter to ensure it is less than 1.

Let S be the set of all the top points of the perimiter. If the value of the tide at hour $n$, $l_n$ is less than the minimal  $z$ value in $q\in S$ then $p_n = 0$. 

Else, let $q' = (x',y', z')$ be the first a point such that for all $q = (x,y,z)$ , $x < x', z > t_n$ and $z' < t_n$. It is the first point underwater. Then find the angle $\theta \in (0, \pi/2)$ from the origin to $(0,r)$ and $(x',y')$.
$$\theta = \cos^{-1}((2r^2 - ({x'}^2+(y' - r)^2)^{\frac{1}{2}})/(2r^2))$$
and
$$p_n = \frac{\pi r \cdot \frac{\theta}{\pi\frac{1}{2}}}{25\pi}$$

$C_{n}$:
$$C_0 = 0, C_{n+1} = \begin{cases} C_n - (C_n \cdot \alpha \cdot p_n) + (F_n \cdot \alpha \cdot p_n) & \text{if $p_n > 0$} \\
0 & \text{if $p_n = 0$}
\end{cases}$$
$F_n$:
$$F_{n} = N - C_{n}$$

$T_n$:

$$T_0 = 0, T_{n+1} = \begin{cases} T_n & \text{if $p_n >0$}\\
T_n + C_n & \text{if $p_n = 0$}
\end{cases}$$

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn
from mpl_toolkits.mplot3d import Axes3D
import os

In [ ]:
%matplotlib notebook

plt3d = plt.figure().gca(projection='3d')


# create x,y
xx, yy = np.meshgrid(range(-25, 30), range(-25,30))

# calculate corresponding z
zz = (6 - (0.17 * yy))

# plot the surface

beach_surf = plt3d.plot_surface(xx, yy, zz, alpha=0.2, color = 'brown', label = "beach")
# tide_surf equations below get the legend to show
beach_surf._facecolors2d=beach_surf._facecolors3d
beach_surf._edgecolors2d=beach_surf._edgecolors3d

h = 2
r = 25
delta = 5

theta = np.linspace(0, np.pi, 100)
x = r * np.cos(theta)
y = r * np.sin(theta) + delta
z = 6 + h - (0.17 * y)

plt3d.plot(x,y,z, label = "top of trap")


tide_path = os.path.join('resources', 'comox_tide.csv')
tide_df = pd.read_csv(tide_path)
tide_df = tide_df.drop(columns = ['PDT'])
tide_values = tide_df.values.flatten()

max_z = max(tide_values)
min_z = min(tide_values)

max_zz = np.full((55,55), max_z)
min_zz = np.full((55,55), min_z)

tide_surf = plt3d.plot_surface(xx, yy, max_zz, color = 'lightcyan', label = "high/low tide")
# tide_surf equations below get the legend to show
tide_surf._facecolors2d=tide_surf._facecolors3d
tide_surf._edgecolors2d=tide_surf._edgecolors3d
plt3d.plot_surface(xx, yy, min_zz, color = 'lightcyan')

plt3d.set_xlabel('X')
plt3d.set_ylabel('Y')
plt3d.set_zlabel('Z')
plt3d.set_xlim(-20,30)
plt3d.set_ylim(-20,30)
plt3d.legend()
plt3d.set_title('fish trap')


In [ ]:
# find the angle from the origin to the (0,r) point and the leading edge covered by water
def circ_covered(tide_level, z_values, x_values, y_values, radius):
    index = -1
    for i in range(len(z_values)):
        if(z_values[i] <= tide_level):
            index = i
            break;
    if(index == -1):
        return 0
    
    x = x_values[index]
    y = y_values[index]
    length = np.sqrt((x)**2 + (y - radius - delta)**2)
    
    angle = math.acos((2 * radius**2 - length**2) / (2 * radius**2))
    coverage  = angle/ (0.5 * np.pi)
    return coverage

In [ ]:
## read in tide values that formed the above tide graph

tide_path = os.path.join('resources', 'comox_tide.csv')
tide_df = pd.read_csv(tide_path)
tide_df = tide_df.drop(columns = ['PDT'])
tide_values = tide_df.values.flatten()

In [ ]:
alpha = 0.05
N = 1000
F = N
C = 0
total_caught = [0]
in_trap = [0]
out_trap = [N]
catches = []
perimeter_ratio =  np.pi * r / (np.pi * 25)

for level in tide_values:
    coverage = circ_covered(level, z, x, y, r)
    free_to_captured = F * coverage * alpha * perimeter_ratio
    captured_to_free = C * coverage * alpha * perimeter_ratio
    C = C - captured_to_free + free_to_captured
    F = F + captured_to_free - free_to_captured
    
    if(coverage > 0):
        total_caught.append(total_caught[-1])
    else:
        total_caught.append(total_caught[-1] + C)
        if(C != 0):
            catches.append(C)
        C = 0
        F = N
        
    
    in_trap.append(C)
    out_trap.append(F)
        

In [ ]:
%matplotlib notebook
## plot the weeks numbers
seaborn.set()
plt.style.use('seaborn-deep')
x_values = range(len(tide_values) + 1)
plt.plot(x_values, in_trap, label = "fish in trap")
plt.plot(x_values, out_trap, label = "fish outside of trap")
plt.plot(x_values, total_caught, label = "total caught")
plt.ylabel("number of fish")
plt.xlabel("time (h)")
plt.title('fish')
plt.legend()
plt.show()

In [ ]:
#The individual catches when the water level lowers as to trap the fish in the semi circle
catches